In [1]:
import pandas as pd
import pandasql as ps
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import jarque_bera

from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("../src/data/EXTR_RPSale.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
q1 = ("""
SELECT * 
FROM df
WHERE DocumentDate like "%2019"
""")
df2019 =ps.sqldf(q1)

In [ ]:
# df2019.to_csv('../references/df2019',index=False)

In [ ]:
df2019 = pd.read_csv("../references/df2019.csv")
df2019.head()

In [ ]:
df = pd.read_csv("../src/data/EXTR_LookUp.csv")
list(df['LUDescription'][df['LUType']==1])

In [ ]:
df[df['LUType']==1].head(8)

In [ ]:
q2 = ("""
SELECT * 
FROM df2019
WHERE (PropertyType = 11 or PropertyType = 12 or PropertyType = 13 or PropertyType = 14) and SalePrice > 0
""")
dfresidence =ps.sqldf(q2)
dfresidence

In [ ]:
dfresidence.columns

In [ ]:
dfresidence['MajorMinor'] = dfresidence ['Major'].astype(str)+'-'+dfresidence ['Minor'].astype(str)
dfResBldg  = pd.read_csv("../src/data/EXTR_ResBldg.csv", encoding='latin-1')

In [ ]:
dfResBldg['MajorMinor'] =dfResBldg ['Major'].astype(str)+'-'+dfResBldg ['Minor'].astype(str)


In [ ]:
df_final = pd.merge(dfresidence,dfResBldg,how='left', on = ['MajorMinor'])
df_final.columns

In [ ]:
df_final = df_final[['MajorMinor','SalePrice','PropertyType','NbrLivingUnits','Address','BuildingNumber','StreetName', 'StreetType','ZipCode','Stories','BldgGrade', 'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor',
       'SqFt2ndFloor', 'SqFtUpperFloor', 'SqFtUnfinFull',
       'SqFtTotLiving', 'SqFtTotBasement', 'SqFtFinBasement',
       'FinBasementGrade', 'SqFtGarageBasement', 'SqFtGarageAttached',
       'DaylightBasement', 'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck',
       'HeatSystem', 'HeatSource', 'ViewUtilization', 'Bedrooms',
       'BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'FpSingleStory',
       'FpMultiStory', 'FpFreestanding', 'YrBuilt',
       'YrRenovated','Condition']]

In [ ]:
df_final

In [ ]:
df_final = df_final.dropna(subset=['Address'])


In [ ]:
corr = df_final.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))

fig1, ax1 = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask, ax=ax1)

In [ ]:
dfParcel  = pd.read_csv("../src/data/EXTR_Parcel.csv", encoding='latin-1')
dfParcel['MajorMinor'] = dfParcel['Major'].astype(str)+'-'+dfParcel['Minor'].astype(str)
dfParcel.columns

In [ ]:
dfParcel = dfParcel[['MajorMinor','WfntLocation','SqFtLot','TrafficNoise', 'AirportNoise', 'PowerLines','OtherNuisances']]

In [ ]:
dfAll = pd.merge(df_final,dfParcel,how='left', on = ['MajorMinor'])
dfAll['WfntLocation'].describe()

In [ ]:
dfAll.describe()

In [ ]:
corr = dfAll.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))

fig1, ax1 = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask, ax=ax1);

In [ ]:
dfAll.columns


In [ ]:
# dfAll.to_csv('../references/dfAll.csv',index=False)

In [ ]:
q3 = ("""
SELECT *
FROM dfAll
WHERE YrRenovated = 2019
""")
ps.sqldf(q3)



In [ ]:
ols(formula="SalePrice ~ SqFtTotLiving", data=dfAll).fit().summary()


In [ ]:
fsm_df = dfAll[['SalePrice','SqFtTotLiving']]
fsm_df.dropna(inplace=True)

In [ ]:
fsm = ols(formula="SalePrice ~ SqFtTotLiving", data=fsm_df).fit()

In [ ]:
fsm.summary()

In [ ]:
qp = ("""
SELECT SalePrice,
CASE
    WHEN SqFtOpenPorch > 0 OR SqFtEnclosedPorch > 0 OR SqFtDeck > 0 THEN 1
    ELSE 0
END AS porch
FROM dfAll
""")
fsm2_df = ps.sqldf(qp)

In [ ]:
fsm2_df

In [ ]:
fsm2 = ols(formula="SalePrice ~ porch", data=fsm2_df).fit()

In [ ]:
fsm2.summary()

In [ ]:
qwf = ("""
SELECT SalePrice,
CASE
    WHEN WfntLocation > 0 THEN 1
    ELSE 0
END as wfnt
FROM dfAll
""")
fsm3_df = ps.sqldf(qwf)

In [ ]:
fsm3_df['wfnt'].value_counts()

In [ ]:
fsm3 = ols(formula="SalePrice ~ wfnt", data=fsm3_df).fit()
fsm3.summary()

In [ ]:
qlp = ("""
SELECT SalePrice, (SqFtTotLiving/SqFtLot) AS lot_proportion

FROM dfAll
""")
lot_prop = ps.sqldf(qlp)
lot_prop

In [ ]:
qcs = ("""
SELECT SalePrice, (SalePrice/SqFtTotLiving) AS cost_sqft
FROM dfAll
""")
ps.sqldf(qcs)


In [ ]:
qall = ("""
SELECT SalePrice, SqFtTotLiving AS square_footage,
CASE
    WHEN SqFtOpenPorch > 0 OR SqFtEnclosedPorch > 0 OR SqFtDeck > 0 THEN 1
    ELSE 0
END AS has_porch,
CASE
    WHEN WfntLocation > 0 THEN 1
    ELSE 0
END as has_waterfront,
(SqFtTotLiving/SqFtLot) AS lot_proportion,
(SalePrice/SqFtTotLiving) AS cost_sqft,
CASE 
    WHEN TrafficNoise > 0 OR AirportNoise > 0 OR PowerLines is 'Y' THEN 1
    ELSE 0
END AS has_nuisance

FROM dfAll
""")
fsm_dfinal = ps.sqldf(qall)



In [ ]:
fsm_dfinal.columns

In [ ]:
#0 SalePrice
#1 square_footage
#2 has_porch
#3 has_waterfront
#4 lot_proportion
#5 cost_sqft
#6 has_nuisance

In [ ]:
# 123
fsm_123 = ols(formula="SalePrice ~ square_footage + has_porch + has_waterfront", data=fsm_dfinal).fit()
d_123 = fsm_dfinal[['square_footage', 'has_porch', 'has_waterfront']]
# 124
fsm_124 = ols(formula="SalePrice ~ square_footage + has_porch + lot_proportion", data=fsm_dfinal).fit()
d_124 = fsm_dfinal[['square_footage', 'has_porch', 'lot_proportion']]
# 125
fsm_125 = ols(formula="SalePrice ~ square_footage + has_porch + cost_sqft", data=fsm_dfinal).fit()
d_125 = fsm_dfinal[['square_footage', 'has_porch', 'cost_sqft']]
# 126
fsm_126 = ols(formula="SalePrice ~ square_footage + has_porch + has_nuisance", data=fsm_dfinal).fit()
d_126 = fsm_dfinal[['square_footage', 'has_porch', 'has_nuisance']]
# 134
fsm_134 = ols(formula="SalePrice ~ square_footage + has_waterfront + lot_proportion", data=fsm_dfinal).fit()
d_134 = fsm_dfinal[['square_footage', 'has_waterfront', 'lot_proportion']]
# 135
fsm_135 = ols(formula="SalePrice ~ square_footage + has_waterfront + cost_sqft", data=fsm_dfinal).fit()
d_135 = fsm_dfinal[['square_footage', 'has_waterfront', 'cost_sqft']]
# 136
fsm_136 = ols(formula="SalePrice ~ square_footage + has_waterfront + has_nuisance", data=fsm_dfinal).fit()
d_136 = fsm_dfinal[['square_footage', 'has_waterfront', 'has_nuisance']]
# 145
fsm_145 = ols(formula="SalePrice ~ square_footage + lot_proportion + cost_sqft", data=fsm_dfinal).fit()
d_145 = fsm_dfinal[['square_footage', 'lot_proportion', 'cost_sqft']]
# 146
fsm_146 = ols(formula="SalePrice ~ square_footage + lot_proportion + has_nuisance", data=fsm_dfinal).fit()
d_146 = fsm_dfinal[['square_footage', 'lot_proportion', 'has_nuisance']]
# 156
fsm_156 = ols(formula="SalePrice ~ square_footage + cost_sqft + has_nuisance", data=fsm_dfinal).fit()
d_156 = fsm_dfinal[['square_footage', 'cost_sqft', 'has_nuisance']]
# 234
fsm_234 = ols(formula="SalePrice ~ has_porch + has_waterfront + lot_proportion", data=fsm_dfinal).fit()
d_234 = fsm_dfinal[['has_porch', 'has_waterfront', 'lot_proportion']]
# 235
fsm_235 = ols(formula="SalePrice ~ has_porch + has_waterfront + cost_sqft", data=fsm_dfinal).fit()
d_235 = fsm_dfinal[['has_porch', 'has_waterfront', 'cost_sqft']]
# 236
fsm_236 = ols(formula="SalePrice ~ has_porch + has_waterfront + has_nuisance", data=fsm_dfinal).fit()
d_236 = fsm_dfinal[['has_porch', 'has_waterfront', 'has_nuisance']]
# 245
fsm_245 = ols(formula="SalePrice ~ has_porch + lot_proportion + cost_sqft", data=fsm_dfinal).fit()
d_245 = fsm_dfinal[['has_porch', 'lot_proportion', 'cost_sqft']]


fsm_246 = ols(formula="SalePrice ~ has_porch + lot_proportion + has_nuisance", data=fsm_dfinal).fit()
d_246 = fsm_dfinal[['has_porch', 'lot_proportion', 'has_nuisance']]
#R^2 = 0.028
fsm_256 = ols(formula="SalePrice ~ has_porch + cost_sqft + has_nuisance", data=fsm_dfinal).fit()
d_256 = fsm_dfinal[['has_porch', 'cost_sqft', 'has_nuisance']]
#R^2 = 0.271
fsm_345 = ols(formula="SalePrice ~ has_waterfront + lot_proportion + cost_sqft", data=fsm_dfinal).fit()
d_345 = fsm_dfinal[['has_waterfront', 'lot_proportion', 'cost_sqft']]
#R^2 = 0.289
fsm_346 = ols(formula="SalePrice ~ has_waterfront + lot_proportion + has_nuisance", data=fsm_dfinal).fit()
d_346 = fsm_dfinal[['has_waterfront', 'lot_proportion', 'has_nuisance']]
#R^2 = 0.107
fsm_356 = ols(formula="SalePrice ~ has_waterfront + cost_sqft + has_nuisance", data=fsm_dfinal).fit()
d_356 = fsm_dfinal[['has_waterfront', 'cost_sqft', 'has_nuisance']]
#R^2 = 0.281
fsm_456 = ols(formula="SalePrice ~ lot_proportion + cost_sqft + has_nuisance", data=fsm_dfinal).fit()
d_456 = fsm_dfinal[['lot_proportion', 'cost_sqft', 'has_nuisance']]
#R^2 = 0.246

In [ ]:
# OLS Regression
def olsreg(fsm):
    return (fsm).summary()

In [ ]:
olsreg(fsm_345)

In [ ]:
# Linearity 
def rainbow(fsm):
    rainbow_statistic, rainbow_p_value = linear_rainbow(fsm)
    print("Rainbow statistic:", rainbow_statistic)
    print("Rainbow p-value:", rainbow_p_value)

In [ ]:
rainbow(fsm_345)

In [ ]:
# overall model fit summary
def overallmodfit(fsm,d):
    print("R^2: " ,fsm.rsquared) # R^2
    print("Adj. R^2: ",fsm.rsquared_adj) # Adj. R^2
    print("F-statistic: ",fsm.fvalue) # F-statistic
    print("Prob(F-statistic)/p-value: ",fsm.f_pvalue) # Prob(F-statistic)/p-value
    print('linearity')
    rainbow_statistic, rainbow_p_value = linear_rainbow(fsm)
    print("Rainbow statistic:", rainbow_statistic)
    print("Rainbow p-value:", rainbow_p_value)
    y = fsm_dfinal['SalePrice']
    y_hat = fsm_246.predict()
    print('homoscedasticity: Breusch Pagan test')
    lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(y-y_hat, d)
    print("Lagrange Multiplier p-value:", lm_p_value)
    print("F-statistic p-value:", f_p_value)
    print('normality')
    jb, jbpv, skew, kurtosis = jarque_bera(y-y_hat)
    print('jarque_bera: ',jb)
    print('jarque_bera p-value: ',jbpv)

In [ ]:
overallmodfit(fsm_124,d_124)